In [64]:
import requests
import re
import json
import csv
import os
from bs4 import BeautifulSoup

def crawling_product(result_folder_path, product_id):
    user_agent = 'PostmanRuntime/7.29.2'
    headers = {
    'User-Agent': user_agent
    }

    url = 'https://tiki.vn/api/v2/products/' + str(product_id)
    response = requests.get(url, headers=headers)

    raw_data = json.loads(response.content.decode())

    needed_attributes = ['id','master_id','sku','name','short_url'
    ,'type','short_description','price','list_price','original_price'
    ,'discount','discount_rate','rating_average','review_count','inventory_status'
    ,'inventory_type','productset_group_name','seller','has_buynow','day_ago_created'
    ,'all_time_quantity_sold','warranty_policy','authors','current_seller','other_sellers'
    ,'price_comparison','specifications','categories','breadcrumbs']
    
    selected_data = {}
    for k,v in raw_data.items():
        if k in needed_attributes:
            selected_data[k] = raw_data[k]

    flaten_data = {}
    products_table = []
    categories_table = []
    categories_url = []
    
    for i in range(len(selected_data['breadcrumbs']) - 1):
        category_id = selected_data['breadcrumbs'][i]['category_id']
        categories_url.append(selected_data['breadcrumbs'][i]['url'])
        
        draft_product_data = {}
        for k,v in selected_data.items():
            if k != 'breadcrumbs':
                draft_product_data[k] = selected_data[k]
        draft_product_data['category_id'] = category_id
        products_table.append(draft_product_data)

        draft_category_data = {}
        for k,v in selected_data['breadcrumbs'][i].items():
            draft_category_data['category_id'] = category_id
            if k != 'category_id':
                draft_category_data[k] = selected_data['breadcrumbs'][i][k]
        categories_table.append(draft_category_data)

    products_fileName = 'products_data.csv'
    products_filePath = result_folder_path + products_fileName
    # '../04_crawling_ws/00_draft/test1.json'

    for item in range(len(products_table)):
        if not os.path.isfile(products_filePath):
            with open(products_filePath, 'a') as storageFile:
                csv_writer = csv.writer(storageFile, delimiter=",")
                csv_writer.writerow(products_table[item].keys())
                csv_writer.writerow(products_table[item].values())
        else:
            with open(products_filePath, 'a') as storageFile:
                csv_writer = csv.writer(storageFile, delimiter=",")
                csv_writer.writerow(products_table[item].values())


    categories_fileName = 'categories_data.csv'
    categories_filePath = result_folder_path + categories_fileName
    print(categories_filePath)
    for item in range(len(categories_table)):
        if not os.path.isfile(categories_filePath):
            with open(categories_filePath, 'a') as storageFile:
                csv_writer = csv.writer(storageFile, delimiter=",")
                csv_writer.writerow(categories_table[item].keys())
                csv_writer.writerow(categories_table[item].values())
        else:
            with open(categories_filePath, 'a') as storageFile:
                csv_writer = csv.writer(storageFile, delimiter=",")
                csv_writer.writerow(categories_table[item].values())

    # return a list of categories url
    return categories_url

In [66]:
def category_mining(category_url):
    user_agent = 'PostmanRuntime/7.29.2'
    headers = {
    'User-Agent': user_agent
    }
    url = 'https://tiki.vn' + str(category_url)
    response = requests.get(url, headers=headers)

    html_source = response.content.decode()
    soup = BeautifulSoup(html_source, 'html.parser')
    
    productID_list = []
    for item in soup.find_all('a'):
        if item.get('data-view-content') != None:
            item_dict = json.loads(item.get('data-view-content'))
            if 'id' in item_dict['click_data'].keys():
                productID_list.append(item_dict['click_data']['id'])

    # return a list of product id
    return productID_list



In [67]:
def process_controller(entry_url):
    id_container = re.findall('-p.*.html', entry_url)[0]
    product_id = id_container.split('-p')[1].split('.html')[0]
    print(product_id)
    
    crawling_product('path', product_id)

    category_id = ''
    category_mining(category_id)

    # need to implement a recursive here

In [ ]:
# test ####################################################################

# crawling_product('../04_crawling_ws/00_draft/', 15445963) 3304875 68585576
crawling_product('../04_crawling_ws/00_draft/', 3304875)

In [ ]:
category_mining('/nha-sach-tiki/c8322')